In [1]:
import nltk
from nltk.tokenize import word_tokenize 
import pandas as pd
import os
import json
import glob
import spacy
nlp = spacy.load('pt')



jsons=[]
for filename in glob.iglob("D:\\socialbus\\users\\*" + '*', recursive=True):
    try:
        jsons.append(pd.read_json(filename,lines=True))
        #print(filename)
        #print("deu")
    except:
        pass
        
print(str(len(jsons)) + " json files found")  

30 json files found


In [39]:
    i=0
    tweets=[]
    
    for json in jsons:
        i=len(json)-1
        while(i>1):
        
            try:
          
                t1=Tweet(json['user'][i]['screen_name'],json['text'][i],get_sentiment_polarity(json['text'][i]),get_time_of_comment(json['created_at'][i]),
                  get_stopword_nr(json['text'][i]),get_nr_of_hashtags(json['metadata'][i]),
                    get_nr_of_mentions(json['metadata'][i]),get_nr_of_urls(json['metadata'][i]),
                  get_nr_of_emoticons(json['metadata'][i]),get_swear_count(json['text'][i]),
                  get_favorites_count(json['favorite_count'][i]),get_verb_count(json['text'][i]),
                  get_noun_count(json['text'][i]),get_adj_count(json['text'][i]),
                  get_tweet_length(json['text'][i]),get_allcaps_to_word_ratio(json['text'][i]))
            
                  
        
                tweets.append(t1)
                break
            
            except:
        
                t1=Tweet(json['user'][i]['screen_name'],json['text'][i],get_sentiment_polarity(json['text'][i]),get_time_of_comment(json['created_at'][i]),
                  get_stopword_nr(json['text'][i]),get_nr_of_hashtags(json['entities'][i]),
                  get_nr_of_mentions(json['entities'][i]),get_nr_of_urls(json['entities'][i]),
                  get_nr_of_emoticons(json['entities'][i]),get_swear_count(json['text'][i]),
                  get_favorites_count(json['favorite_count'][i]),get_verb_count(json['text'][i]),
                  get_noun_count(json['text'][i]),get_adj_count(json['text'][i]),
                  get_tweet_length(json['text'][i]),get_allcaps_to_word_ratio(json['text'][i]))
                
        
                tweets.append(t1)
                break
            
        
            i=i-1 
           

    
    #data=pd.Series(tweets)
    create_dataFrame(tweets)
        




RT @eujafizaquiloou: Filipinos brancos ou negros

Negros-fav

Brancos-Rt
0


KeyboardInterrupt: 

In [ ]:
def get_stopword_nr(sentence):
    
    word_tokens = word_tokenize(sentence)
    count=0
    
    stopwords = nltk.corpus.stopwords.words('portuguese')
    for stopword in stopwords:
        for word in word_tokens:
            if stopword==word:
                count+=1
       
    
    return count
    
    

In [ ]:
def get_time_of_comment(jsonLine):
     
    dateStr=str(jsonLine)
       
    fullDateArray=dateStr.split()
    time=fullDateArray[1].split('+')
    
    return time[0]
    

In [ ]:
def get_nr_of_hashtags(jsonLine):
    return len(jsonLine['hashtags'])

In [ ]:
def get_nr_of_mentions(jsonLine):
    try:
        mentions=len(jsonLine['mentions'])
        return mentions
    except:
        mentions=len(jsonLine['user_mentions'])
        return mentions

In [ ]:
def get_nr_of_urls(jsonLine):
    return len(jsonLine['urls'])

In [ ]:
def get_nr_of_emoticons(jsonLine):
    try:
        emoticons= len(jsonLine['emoticons'])
        return emoticons
    except:
        emoticons=len(jsonLine['symbols'])

In [ ]:
def get_swear_count(jsonLine):
    swearCount=0
    swears=['broch','cabrões','cabr','cagalhão','caralh','cona','corn','enrabar','cuzinho','esporra','fod','mamadas','panasca','peid','piça','picha','pila','tomates','puta','merd','caralho','crl','fds','fdç','paneleir']
    for swear in swears:
        if swear.lower() in jsonLine.lower():
            swearCount+=1
            
    return swearCount        

In [ ]:
def get_favorites_count(jsonLine):
    return jsonLine

In [ ]:
def get_day_of_tweet(jsonLine):
    dateStr=str(jsonLine)
       
    fullDateArray=dateStr.split()

In [ ]:
def get_verb_count(jsonLine):
    sentence=jsonLine
    doc = nlp(sentence)

    verbCount=0


    for token in doc:
        if token.pos_=='VERB':
            verbCount+=1
        
    return verbCount

In [ ]:
def get_noun_count(jsonLine):
    sentence=jsonLine
    doc = nlp(sentence)

    nounCount=0


    for token in doc:
        if token.pos_=='NOUN' or token.pos=='PROPN':
            nounCount+=1
        
    return nounCount

In [ ]:
def get_adj_count(jsonLine):
    sentence=jsonLine
    doc = nlp(sentence)

    adjCount=0


    for token in doc:
        if token.pos_=='ADJ':
            adjCount+=1
        
    return adjCount

In [ ]:
def get_tweet_length(jsonLine):
    return len(jsonLine)

In [ ]:
import sentimentFlex

def get_sentiment_polarity(jsonLine):
  
    sentilex = sentimentFlex.sentiLexFlexToDict() # dict to use to know the sentiment
    sentimentValue = sentimentFlex.polarity(sentilex,jsonLine)
    
 
    return sentimentValue

In [ ]:
def create_dataFrame(tweets):
   # columns=['Username','Text','Time of Comment','Number of Stopwords','Number of Hashtags','Number of Mentions','Number of Urls',
            # 'Number of Emoticons','Number of Swears','Number of Favorites','Number of Verbs','Number of Nouns','Number of Adjectives','Tweet Length']
   

    #df= pd.DataFrame.from_records([tweets.to_dict() for t in tweets])
    
    variables = tweets[0].to_dict().keys()
    df = pd.DataFrame([[getattr(i,j) for j in variables] for i in tweets], columns = variables)
    
    print(df)
    
    return df.to_csv('firstcsv.csv',sep='\t', encoding='utf-8')

In [38]:
import re

def get_allcaps_to_word_ratio(jsonLine):
  
   
    sentence=jsonLine.split()
    matches = re.findall("([A-Z]+\s?[A-Z]+[^a-z0-9\W])",jsonLine)

    if(sentence>0):
        ratio= len(matches)/len(sentence)
        return ratio
    else:
        return 0

0.3333333333333333


In [ ]:
class Tweet:
      
        
        def __init__(self, username,text,sentimentPolarity, time_of_comment, nrStopwords, nrHashtags, nrMentions, nrUrls, nrEmoticons, nrSwears, nrFavs,
                  nrVerbs,nrNouns,nrAdjectives,tweetLength,allCapsToWordRatio):
            self.username=username
            self.text=text
            self.sentimentPolarity=sentimentPolarity
            self.time_of_comment=time_of_comment
            self.nrStopwords=nrStopwords
            self.nrHashtags=nrHashtags
            self.nrMentions=nrMentions
            self.nrUrls=nrUrls
            self.nrEmoticons=nrEmoticons
            self.nrSwears=nrSwears
            self.nrFavs=nrFavs
            self.nrVerbs=nrVerbs
            self.nrNouns=nrNouns
            self.nrAdjectives=nrAdjectives
            self.tweetLength=tweetLength
            self.allCapsToWordRatio=allCapsToWordRatio
            
            
            
        def __str__(self):
            return self.username + " " + self.text +" " +self.time_of_comment
        
        def to_dict(self):
            return {
            'username':self.username,
            'text':self.text,
            'time_of_comment': self.time_of_comment,
            'sentimentPolarity': self.sentimentPolarity,
            'tweetLength':self.tweetLength,
            'nrStopwords': self.nrStopwords,
            'nrHashtags':self.nrHashtags,
            'nrMentions':self.nrMentions,
            'nrUrls':self.nrUrls,
            'nrEmoticons':self.nrEmoticons,
            'nrSwears':self.nrSwears,
            'nrFavs':self.nrFavs,
            'nrVerbs':self.nrVerbs,
            'nrNouns':self.nrNouns,
            'nrAdjectives':self.nrAdjectives,
            'allCapsToWordRatio':self.allCapsToWordRatio
            }
            
            
            
            
            
            
            
            
        